## LLM 도구 바인딩(Binding Tools)

LLM 모델이 도구를 호출할 수 있으려면 chat 요청 시 모델에 도구 스키마(tool schema)를 포함해야 함.<br>
도구 호출(tool calling) 기능을 지원하는 LangChain Chat Model은 .bind_tools() 메서드를 구현하여 LangChain 도구 객체, Pydantic 클래스 또는 <br>JSON 스키마 목록을 수신하고 공급자별 예상 형식으로 채팅 모델에 바인딩.<br>
바인딩된 Chat Model의 후속 호출은 모델 API에 대한 모든 호출에 도구 스키마를 포함.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("Bind-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
Bind-Tools


### Binding할 Tool 정의
- get_word_length: 단어의 길이를 반환하는 함수
- add_function: 두 숫자를 더하는 함수
- naver_news_crawl: 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

참고
- 도구를 정의할 때 @tool 데코레이터를 사용하여 도구 정의
- docstring은 가급적 영어로 작성하는 것을 권장

In [2]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

# 도구를 정의
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

In [4]:
# 결과 tool_calls에 저장. .tool_calls를 확인하여 도구 호출 결과 확인 가능.
llm_with_tools.invoke("What is the length of the word 'teddynote'?").tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'teddynote'},
  'id': 'call_htzzg7Q3Knt2h7PfgYB4nrVG',
  'type': 'tool_call'}]

In [7]:
from langchain_core.output_parsers import JsonOutputToolsParser

chain = llm_with_tools | JsonOutputToolsParser()
tool_call_result = chain.invoke("What is the length of the word 'teddynote'?")

In [8]:
tool_call_result

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]

In [9]:
tool_call_result[0]

{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}

In [10]:
def excute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수.
    :param tool_call_result: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    for tool_call_result in tool_call_results:
        # 도구 이름과 인자 추출
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]
        # 도구 이름과 일치하는 도구 찾아 실행
        # next() 함수를 사용하여 일치하는 첫번째 도구 찾기.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구 찾았다면 해당 도구 실행
            result = matching_tool.invoke(tool_args)
            # 실행 결과 출력.
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            print(f"도구 이름과 일치하는 도구를 찾을 수 없습니다: {tool_name}")

In [11]:
excute_tool_calls(tool_call_result)

[실행도구] get_word_length
[실행결과] 9


### bind_tools + Parser + Execution

위의 과정을 한번에 실행
- llm_with_tools: 도구를 바인딩한 모델
- JsonOutputToolsParser(): 도구 호출 결과를 JSON 형식으로 파싱
- excute_tool_calls(): 도구 호출 결과를 실행하는 함수

흐름 정리
- 모델에 도구 바인딩
- 도구 호출 결과 파싱
- 도구 호출 결과 실행

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

chain = llm_with_tools | JsonOutputToolsParser() | excute_tool_calls

In [13]:
# 실행 결과
chain.invoke("What is the length of the word 'teddynote'?")

[실행도구] get_word_length
[실행결과] 9


In [15]:
chain.invoke("114.5 + 121.2")

[실행도구] add_function
[실행결과] 235.7


### bind_tools > Agent & AgentExecutor로 대체
bind_tools()는 모델에 사용할 수 있는 스키마(도구) 제공.
AgentExecutor는 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성.

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but don't know current events."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [17]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length, add_function, naver_news_crawl]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools = tools,
    verbose = True,
    handle_parse_errors = True,
)                           

In [18]:
result = agent_executor.invoke({"input": "How many letters in the word 'teddynote'?"})

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'teddynote'}`


9The word 'teddynote' has 9 letters.

> Finished chain.
The word 'teddynote' has 9 letters.


In [19]:
result2 = agent_executor.invoke({"input": "114.5 + 121.2"})

print(result2["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7The sum of 114.5 and 121.2 is 235.7.

> Finished chain.
The sum of 114.5 and 121.2 is 235.7.


In [20]:
result3 = agent_executor.invoke({"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"})

print(result3["output"])
print("=============\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3
Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


In [21]:
result4 = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result4["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`


[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국